In [ ]:
# Cell 1: Imports
import sys
import os
import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import classification_report, accuracy_score

# Cell 2: Parameters
# tags=["parameters"]
INPUT_SEMI_PATH = '../data/processed/dataset_semi_supervised.parquet'
METRICS_OUT = '../data/processed/semi_metrics.json'

# Cell 3: Main Execution
print("--- BẮT ĐẦU: SELF-TRAINING ---")

# 1. Load Data
if not os.path.exists(INPUT_SEMI_PATH):
    raise FileNotFoundError("Chưa chạy bước semi_dataset_preparation!")

data = joblib.load(INPUT_SEMI_PATH)
X_train = data['X_train']
y_train_semi = data['y_train_semi'] # Chứa giá trị -1
X_test = data['X_test']
y_test = data['y_test']

print("Dữ liệu đã load thành công.")
print(f"Số lượng mẫu Unlabeled (-1) trong train: {sum(y_train_semi == -1)}")

# 2. Thiết lập mô hình cơ sở (Base Estimator)
base_clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)

# 3. Thiết lập Self-Training
# threshold=0.75: Chỉ tin tưởng gán nhãn nếu xác suất dự báo > 75%
self_training_model = SelfTrainingClassifier(base_clf, threshold=0.75, criterion='threshold')

# 4. Huấn luyện
print("Đang huấn luyện Self-Training (Quá trình này có thể lâu)...")
self_training_model.fit(X_train, y_train_semi)

print(f"Số vòng lặp thực hiện: {self_training_model.n_iter_}")
# termination_condition_ trả về lý do dừng (max_iter hay all_labeled)
# Lưu ý: Các phiên bản sklearn cũ có thể không có thuộc tính này, có thể bỏ qua print nếu lỗi.

# 5. Đánh giá trên tập Test
print("\n--- KẾT QUẢ TRÊN TẬP TEST ---")
y_pred = self_training_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)

print(f"Accuracy: {acc:.4f}")
print("Chi tiết:")
print(classification_report(y_test, y_pred))

# 6. So sánh nhanh với Baseline (Chỉ train trên 10% dữ liệu có nhãn)
print("\n--- SO SÁNH VỚI BASELINE (Supervised trên 10% data) ---")
mask_labeled = y_train_semi != -1
baseline_clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
baseline_clf.fit(X_train[mask_labeled], y_train_semi[mask_labeled])
y_pred_base = baseline_clf.predict(X_test)
acc_base = accuracy_score(y_test, y_pred_base)
print(f"Baseline Accuracy: {acc_base:.4f}")

# 7. Lưu kết quả
import json
results = {
    "self_training_accuracy": acc,
    "baseline_accuracy": acc_base,
    "n_iter": int(self_training_model.n_iter_) if hasattr(self_training_model, 'n_iter_') else 0,
    "classification_report": report
}

with open(METRICS_OUT, 'w') as f:
    json.dump(results, f)
print(f"Đã lưu kết quả tại: {METRICS_OUT}")